## Tensorflow 1.4

In the first step I implemented tensorflow a simple cnn using tf 1.4  

In [ ]:
from cnn_utils import *
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import scipy
from PIL import Image
from scipy import ndimage
import tensorflow as tf
from tensorflow.python.framework import ops

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [ ]:
x_train, y_train, x_test, y_test, classes = load_dataset()

In [ ]:
x_train.shape, y_train.shape, x_test.shape, classes.shape

In [ ]:
idx = 320
plt.imshow(x_train[idx])
print('y={}'.format(y_train[0][idx]))

In [ ]:
x_train, x_test = x_train/255, x_test/255

# build one-hot labels
y_train = np.array(list(map(lambda x: np.eye(len(classes))[int(x[0])], y_train.T)))
y_test = np.array(list(map(lambda x: np.eye(len(classes))[int(x[0])], y_test.T)))

In [ ]:
y_test.shape, y_train.shape

In [ ]:
def forward_propagation(X, params):
    W1, W2 = params['w1'], params['w2']
    
    Z1 = tf.nn.conv2d(X, W1, strides=[1], padding='SAME')
    A1 = tf.nn.relu(Z1)
    P1 = tf.nn.max_pool(A1, ksize=[8], strides=[8], padding='SAME')
    Z2 = tf.nn.conv2d(P1, W2, strides=[1], padding='SAME')
    A2 = tf.nn.relu(Z2)
    P2 = tf.nn.max_pool(A2, ksize=[4], strides=[4], padding='SAME')
    P2 = tf.contrib.layers.flatten(P2)
    Z3 = tf.contrib.layers.fully_connected(P2, 6, activation_fn=None)
    
    return Z3


def comp_loss(Y, labels):
    return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits
                          (logits=Y, labels=labels))


In [ ]:
forward_propagation(X, params)

In [ ]:
def random_mini_batch(X, Y, bs):
    permutation = list(np.random.permutation(X.shape[0]))
    shuffled_x, shuffled_y = X[permutation,:,:,:], Y[permutation,:]
    
    batches = []
    for k in range(X.shape[0]//bs):
        mini_batch_x = shuffled_x[k*bs:(k+1)*bs,:,:,:]
        mini_batch_y = shuffled_y[k*bs:(k+1)*bs,:]
        batches.append((mini_batch_x, mini_batch_y))
    return batches


In [ ]:
costs = []
epochs = 100
with tf.device('/cpu:0'):
    bs = 10
    
    ops.reset_default_graph()

    X = tf.placeholder(tf.float32, name='X', shape=(None, 64, 64, 3))
    Y = tf.placeholder(tf.float32, name='Y', shape=[None, 6])

    with tf.variable_scope('declaration', reuse=tf.AUTO_REUSE):
        w1 = tf.get_variable('w1', shape=(4,4,3,8), initializer=tf.contrib.layers.xavier_initializer())
        w2 = tf.get_variable('w2', shape=(2,2,8,16), initializer=tf.contrib.layers.xavier_initializer())
        params = {'w1': w1, 'w2': w2}

    output = forward_propagation(X, params)
    loss = comp_loss(output, Y)
    optimizer = tf.train.AdamOptimizer(learning_rate=0.005).minimize(loss)

    init = tf.global_variables_initializer()

    saver = tf.train.Saver()
    with tf.Session() as sess:
        sess.run(init)

        for epoch in range(epochs):
            batch_loss = 0
            num_batches = x_train.shape[0]//bs
            batches = random_mini_batch(x_train, y_train, bs)

            for x,y in batches:
                _, bloss = sess.run([optimizer, loss], feed_dict={X: x, Y: y})
                batch_loss += bloss / num_batches

            print("loss in epoch {} is : {}".format(epoch, batch_loss))
            saver.save(sess, './sessoin-tmp.cptk')
            

In [ ]:
plt.plot(costs)

In [ ]:
saver = tf.train.Saver()

with tf.Session() as sess:
    saver.restore(sess,  './sessoin-tmp.cptk')
    
    correct = tf.equal(tf.arg_max(Y,1), tf.arg_max(output,1))
    _acc = tf.reduce_mean(tf.cast(correct, tf.float32))
    train_acc = _acc.eval({X:x_train, Y:y_train})
    test_acc = _acc.eval({X:x_test, Y:y_test})
    
    print("train_acc: {} test_acc: {}".format(train_acc, test_acc))
    
    idx = 12
    
    fig = plt.figure()
    tests = [12, 14, 30, 40]
    for idx, i in enumerate(tests):
        chosen_x = np.array([x_test[i,:,:,:]])
        predicted = tf.arg_max(output, dimension=1)
        res = sess.run([predicted], feed_dict={X:chosen_x})
        
        plt.subplot(1, len(tests), idx+1)
        plt.title("predicted: {}".format(res[0]))
        plt.imshow(x_test[i])
    

## Tensorflow 2.0

In this step i'm gonna implement previous model using keras in tf 2.0

In [ ]:
import tensorflow as tf

tf.__version__
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [ ]:
from cnn_utils import *

x_train, y_train, x_test, y_test, classes = load_dataset()
batch_size = 25

x_train = x_train / 255
x_test = x_test / 255

y_train = y_train.T
y_test = y_test.T


In [ ]:
# batching and shuffling dataset
# initial dimentions should match

train_ds = tf.data.Dataset.from_tensor_slices(
(x_train, y_train)).shuffle(1000).batch(batch_size)

test_ds = tf.data.Dataset.from_tensor_slices(
(x_test, y_test)).batch(batch_size)

print(train_ds, test_ds)

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D


class CnnModel(Model):
    def __init__(self):
        super(CnnModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(6, activation='softmax')
    
    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

model = CnnModel()
    

In [ ]:
loss = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
train_loss = tf.keras.metrics.Mean(name='train_loss')
test_loss = tf.keras.metrics.Mean(name='test_loss')

train_acc = tf.keras.metrics.SparseCategoricalAccuracy(name='train_acc')
test_acc = tf.keras.metrics.SparseCategoricalAccuracy(name='test_acc')

In [ ]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        pred = model(images)
        t_loss = loss(labels, pred)
    gradients = tape.gradient(t_loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(t_loss)
    train_acc(labels, pred)




In [ ]:
@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss(labels, predictions)

  test_loss(t_loss)
  test_acc(labels, predictions)

In [ ]:
epochs = 20

for epoch in range(epochs):
    for images, labels in train_ds:
        train_step(images, labels)

    for images, labels in test_ds:
        test_step(images, labels)

    print('Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'.format(
        epoch + 1,
        train_loss.result(),
        train_acc.result(),
        test_loss.result(),
        test_acc.result()
    ))